# Data Analytics & Machine Learning Foundations
## Healthcare No-Show Prediction System

This notebook implements the complete Week 1-7 curriculum:
- **Week 1**: Data Literacy, CRISP-DM, EDA, Tools Setup
- **Week 2**: SQL for Analytics
- **Week 3**: Python for Data Analysis (pandas, matplotlib, unit testing)
- **Week 4-5**: Supervised Learning (7 Classification Algorithms)
- **Week 6**: Unsupervised Learning (Clustering, PCA, Anomaly Detection)
- **Week 7**: Reinforcement Learning (Q-Learning, Policy Optimization)

---
# Week 1: Data Loading & EDA

In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Change to project root directory
os.chdir(Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent)
print(f"Working directory: {os.getcwd()}")

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ Libraries loaded successfully!")

In [ ]:
# Load configuration
import yaml

# Try multiple config paths
config_paths = [
    "config/config.yaml",           # If running from project root
    "../config/config.yaml",        # If running from notebooks/
    Path.cwd().parent / "config" / "config.yaml"  # Absolute fallback
]

config = None
for config_path in config_paths:
    try:
        with open(config_path, "r") as f:
            config = yaml.safe_load(f)
        print(f"✅ Loaded config from: {config_path}")
        break
    except FileNotFoundError:
        continue

if config is None:
    print("⚠️  Config not found, using defaults")
    config = {
        'project': {'name': 'Healthcare Appointments', 'version': '1.0.0'},
        'business': {'cost_per_noshow': 150}
    }

print(f"Project: {config['project']['name']}")
print(f"Version: {config['project']['version']}")

## 1.1 Data Loading

In [ ]:
# Load data from database
DB_PATH = "healthcare.db"

# Check if database exists
if not Path(DB_PATH).exists():
    # Try notebooks directory path
    DB_PATH = "notebooks/healthcare.db"
    
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query("SELECT * FROM appointments", conn)
print(f"✅ Loaded {len(df):,} appointments from database")
print(f"✅ Columns: {len(df.columns)}")
print(f"✅ Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# First look at the data
df.head()

In [ ]:
# Basic info
print("=" * 50)
print("DATA STRUCTURE")
print("=" * 50)
df.info()

## 1.2 Exploratory Data Analysis (EDA)

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
print("Missing Values:")
print(missing_df[missing_df['Missing'] > 0] if missing.sum() > 0 else "No missing values!")

In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# No-show counts
no_show_counts = df['No_Show'].value_counts()
axes[0].bar(['Showed Up', 'No-Show'], no_show_counts.values, color=['#22c55e', '#ef4444'])
axes[0].set_title('Appointment Outcomes', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate(no_show_counts.values):
    axes[0].text(i, v + 1000, f'{v:,}', ha='center', fontweight='bold')

# No-show rate pie
axes[1].pie([no_show_counts[0], no_show_counts[1]], 
            labels=['Showed Up\n(79.8%)', 'No-Show\n(20.2%)'],
            colors=['#22c55e', '#ef4444'],
            explode=(0, 0.1),
            autopct='%1.1f%%',
            startangle=90)
axes[1].set_title('No-Show Rate Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/figures/noshow_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Age distribution
fig, ax = plt.subplots(figsize=(10, 5))
df['Age'].hist(bins=50, ax=ax, color='#3b82f6', edgecolor='white', alpha=0.8)
ax.axvline(df['Age'].mean(), color='#ef4444', linestyle='--', linewidth=2, label=f'Mean: {df["Age"].mean():.1f}')
ax.axvline(df['Age'].median(), color='#22c55e', linestyle='--', linewidth=2, label=f'Median: {df["Age"].median():.1f}')
ax.set_xlabel('Age', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Age Distribution of Patients', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# No-show rate by age group
age_noshow = df.groupby('Age_Group').agg({
    'No_Show': ['count', 'sum', 'mean']
}).round(4)
age_noshow.columns = ['Total', 'No-Shows', 'Rate']
age_noshow['Rate_Pct'] = (age_noshow['Rate'] * 100).round(2)
print("No-Show Rate by Age Group:")
print(age_noshow)

In [ ]:
# Visualize no-show by age group
fig, ax = plt.subplots(figsize=(10, 5))
age_order = ['Child', 'Teen', 'Young Adult', 'Adult', 'Middle Age', 'Senior', 'Elderly']
age_data = df.groupby('Age_Group')['No_Show'].mean().reindex(age_order) * 100

bars = ax.bar(age_order, age_data, color=['#22c55e' if x < 20 else '#eab308' if x < 22 else '#ef4444' for x in age_data])
ax.axhline(20.19, color='#6b7280', linestyle='--', linewidth=2, label='Overall Rate (20.19%)')
ax.set_xlabel('Age Group', fontsize=12)
ax.set_ylabel('No-Show Rate (%)', fontsize=12)
ax.set_title('No-Show Rate by Age Group', fontsize=14, fontweight='bold')
ax.legend()

for bar, val in zip(bars, age_data):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}%', 
            ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

## 1.3 Feature Engineering Preview

In [ ]:
# Lead time analysis
print("Lead Time Statistics:")
print(df['Lead_Days'].describe())

In [ ]:
# Lead time categories distribution
# Create lead time category from Lead_Days
def categorize_lead_time(days):
    if days == 0:
        return 'Same Day'
    elif days <= 7:
        return '1-7 Days'
    elif days <= 14:
        return '8-14 Days'
    elif days <= 30:
        return '15-30 Days'
    else:
        return '30+ Days'

df['Lead_Time_Category'] = df['Lead_Days'].apply(categorize_lead_time)
lead_cat = df['Lead_Time_Category'].value_counts()
print("\nLead Time Categories:")
print(lead_cat)

---
# Week 2: SQL for Analytics

## 2.1 Overall Performance Metrics

In [ ]:
query1 = """
SELECT 
    COUNT(*) as total_appointments,
    SUM(No_Show) as total_no_shows,
    SUM(showed_up) as total_showed_up,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent,
    COUNT(DISTINCT PatientId) as unique_patients
FROM appointments;
"""
result1 = pd.read_sql_query(query1, conn)
print("📊 Overall Performance Metrics")
print("=" * 50)
display(result1)

## 2.2 Neighborhood Risk Analysis (Window Functions)

In [ ]:
query2 = """
SELECT 
    neighbourhood,
    COUNT(*) as total_appointments,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent,
    RANK() OVER (ORDER BY AVG(No_Show) DESC) as risk_rank
FROM appointments
GROUP BY neighbourhood
HAVING COUNT(*) >= 100
ORDER BY no_show_rate_percent DESC
LIMIT 10;
"""
result2 = pd.read_sql_query(query2, conn)
print("📍 Top 10 High-Risk Neighborhoods")
print("=" * 50)
display(result2)

## 2.3 SMS Reminder Effectiveness (Subqueries)

In [ ]:
query3 = """
SELECT 
    CASE WHEN SMS_received = 1 THEN 'SMS Sent' ELSE 'No SMS' END as sms_status,
    COUNT(*) as total_appointments,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent,
    ROUND(AVG(No_Show) * 100 - 
          (SELECT AVG(No_Show) * 100 FROM appointments), 2) as diff_from_baseline
FROM appointments
GROUP BY SMS_received;
"""
result3 = pd.read_sql_query(query3, conn)
print("📱 SMS Reminder Effectiveness")
print("=" * 50)
display(result3)

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(result3['sms_status'], result3['no_show_rate_percent'], 
              color=['#ef4444', '#22c55e'])
ax.set_ylabel('No-Show Rate (%)')
ax.set_title('Impact of SMS Reminders on No-Show Rate', fontweight='bold')
for bar, val in zip(bars, result3['no_show_rate_percent']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
            f'{val}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

## 2.4 Lead Time Analysis (CASE Statements)

In [ ]:
query4 = """
SELECT 
    CASE 
        WHEN Lead_Days = 0 THEN 'Same Day'
        WHEN Lead_Days BETWEEN 1 AND 7 THEN '1-7 Days'
        WHEN Lead_Days BETWEEN 8 AND 14 THEN '1-2 Weeks'
        WHEN Lead_Days BETWEEN 15 AND 30 THEN '2-4 Weeks'
        ELSE 'Over 1 Month'
    END as lead_time_category,
    COUNT(*) as total_appointments,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent
FROM appointments
GROUP BY lead_time_category
ORDER BY no_show_rate_percent;
"""
result4 = pd.read_sql_query(query4, conn)
print("📅 No-Show Rate by Lead Time")
print("=" * 50)
display(result4)

## 2.5 Day of Week Analysis

In [ ]:
query5 = """
SELECT 
    Appointment_Weekday,
    COUNT(*) as total_appointments,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent,
    RANK() OVER (ORDER BY AVG(No_Show) DESC) as worst_day_rank
FROM appointments
GROUP BY Appointment_Weekday
ORDER BY worst_day_rank;
"""
result5 = pd.read_sql_query(query5, conn)
print("📆 No-Show Rate by Day of Week")
print("=" * 50)
display(result5)

## 2.6 Patient Risk Segmentation (Advanced)

In [ ]:
query6 = """
SELECT 
    CASE 
        WHEN Age < 18 THEN 'Youth'
        WHEN Age >= 60 THEN 'Senior'
        ELSE 'Adult'
    END as age_segment,
    CASE WHEN SMS_received = 1 THEN 'With SMS' ELSE 'No SMS' END as sms_segment,
    CASE WHEN Scholarship = 1 THEN 'Low Income' ELSE 'Regular' END as income_segment,
    COUNT(*) as total_appointments,
    ROUND(AVG(No_Show) * 100, 2) as no_show_rate_percent
FROM appointments
GROUP BY age_segment, sms_segment, income_segment
HAVING COUNT(*) >= 50
ORDER BY no_show_rate_percent DESC
LIMIT 10;
"""
result6 = pd.read_sql_query(query6, conn)
print("🎯 Top 10 Highest-Risk Patient Segments")
print("=" * 50)
display(result6)

---
# Summary Dashboard

In [ ]:
# Create summary dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Overall metrics
ax = axes[0, 0]
metrics = ['Total Appts', 'No-Shows', 'Show Rate', 'Unique Patients']
values = [110527, 22319, 79.81, 62299]
colors = ['#3b82f6', '#ef4444', '#22c55e', '#8b5cf6']
bars = ax.bar(metrics, values, color=colors)
ax.set_title('Overall Performance Metrics', fontweight='bold')
ax.set_ylabel('Count / Percentage')

# 2. Age group comparison
ax = axes[0, 1]
age_rates = df.groupby('Age_Group')['No_Show'].mean() * 100
age_rates.plot(kind='barh', ax=ax, color='#3b82f6')
ax.axvline(20.19, color='#ef4444', linestyle='--', label='Baseline')
ax.set_xlabel('No-Show Rate (%)')
ax.set_title('No-Show Rate by Age Group', fontweight='bold')

# 3. SMS impact
ax = axes[1, 0]
sms_data = df.groupby('SMS_received')['No_Show'].mean() * 100
ax.bar(['No SMS', 'SMS Sent'], sms_data.values, color=['#ef4444', '#22c55e'])
ax.set_ylabel('No-Show Rate (%)')
ax.set_title('SMS Reminder Impact', fontweight='bold')

# 4. Lead time trend
ax = axes[1, 1]
lead_rates = result4.set_index('lead_time_category')['no_show_rate_percent']
ax.plot(lead_rates.index, lead_rates.values, marker='o', linewidth=2, markersize=8, color='#3b82f6')
ax.set_ylabel('No-Show Rate (%)')
ax.set_title('No-Show Rate by Lead Time', fontweight='bold')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('outputs/figures/month1_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Key Findings & Business Recommendations

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║                    MONTH 1 KEY FINDINGS                          ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  📊 BASELINE METRICS                                             ║
║  • 110,527 total appointments analyzed                           ║
║  • 20.19% overall no-show rate                                   ║
║  • 62,299 unique patients                                        ║
║                                                                  ║
║  📱 SMS REMINDERS                                                ║
║  • Reduce no-shows by 3.49 percentage points                     ║
║  • Only 32% currently receive SMS                                ║
║  • Expanding to 100% could save $860K annually                   ║
║                                                                  ║
║  👥 HIGH-RISK SEGMENTS                                           ║
║  • Young adults (18-24): 24.01% no-show rate                     ║
║  • Low-income without SMS: highest risk segment                  ║
║  • Island neighborhoods: transportation barriers                  ║
║                                                                  ║
║  📅 SCHEDULING PATTERNS                                          ║
║  • Same-day: 15.94% vs Over 1 month: 24.03%                      ║
║  • Saturday: lowest no-show rate (14.66%)                        ║
║  • Thursday: highest no-show rate (21.23%)                       ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

---
# Week 3: Python for Data Analysis

## 3.1 Reusable Data Cleaning Functions

In [ ]:
# Define reusable cleaning functions with proper docstrings

def fix_age_outliers(df: pd.DataFrame, max_age: int = 100) -> pd.DataFrame:
    """
    Fix negative and extreme ages in the dataset.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame with 'Age' column
    max_age : int, optional
        Maximum valid age (default 100)
    
    Returns
    -------
    pd.DataFrame
        DataFrame with corrected ages
    
    Examples
    --------
    >>> df = fix_age_outliers(df, max_age=100)
    """
    df = df.copy()
    
    # Fix negative ages
    negative_count = (df['Age'] < 0).sum()
    df.loc[df['Age'] < 0, 'Age'] = df.loc[df['Age'] < 0, 'Age'].abs()
    
    # Cap extreme ages
    extreme_count = (df['Age'] > max_age).sum()
    df.loc[df['Age'] > max_age, 'Age'] = max_age
    
    print(f"✅ Fixed {negative_count} negative ages")
    print(f"✅ Capped {extreme_count} ages > {max_age}")
    
    return df


def create_lead_time(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate days between scheduling and appointment.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with 'ScheduledDay' and 'AppointmentDay' columns
    
    Returns
    -------
    pd.DataFrame
        DataFrame with 'lead_days' column added
    """
    df = df.copy()
    
    df['lead_days'] = (
        pd.to_datetime(df['AppointmentDay']) - 
        pd.to_datetime(df['ScheduledDay'])
    ).dt.days
    
    # Clip negative values
    df['lead_days'] = df['lead_days'].clip(lower=0)
    
    print(f"✅ Lead time range: {df['lead_days'].min()} to {df['lead_days'].max()} days")
    
    return df


def categorize_age(age: int) -> str:
    """
    Categorize age into groups.
    
    Parameters
    ----------
    age : int
        Patient age
    
    Returns
    -------
    str
        Age group category
    """
    if age < 13:
        return 'Child'
    elif age < 18:
        return 'Teen'
    elif age < 25:
        return 'Young Adult'
    elif age < 50:
        return 'Adult'
    elif age < 65:
        return 'Middle Age'
    else:
        return 'Senior'


print("✅ Reusable functions defined!")

## 3.2 Building a Reusable Pipeline

In [ ]:
# Demonstrate the pipeline pattern

def clean_appointments_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    """
    Execute complete data cleaning pipeline.
    
    Parameters
    ----------
    df : pd.DataFrame
        Raw appointments data
    
    Returns
    -------
    pd.DataFrame
        Cleaned and feature-engineered data
    """
    print("=" * 50)
    print("RUNNING DATA CLEANING PIPELINE")
    print("=" * 50)
    
    # Step 1: Fix ages
    df = fix_age_outliers(df, max_age=100)
    
    # Step 2: Create lead time (if columns exist)
    if 'ScheduledDay' in df.columns and 'AppointmentDay' in df.columns:
        df = create_lead_time(df)
    
    # Step 3: Create age groups
    if 'Age' in df.columns:
        df['age_group_new'] = df['Age'].apply(categorize_age)
        print(f"✅ Created age groups: {df['age_group_new'].nunique()} categories")
    
    print("=" * 50)
    print("✅ PIPELINE COMPLETE!")
    print(f"✅ Final shape: {df.shape}")
    
    return df

# Run the pipeline on a sample
df_sample = df.head(1000).copy()
df_cleaned = clean_appointments_pipeline(df_sample)

## 3.3 Unit Testing Example

In [ ]:
# Simple unit test demonstration

def test_fix_age_outliers():
    """Unit test for age outlier function."""
    # Create test data
    test_df = pd.DataFrame({'Age': [25, -5, 150, 30]})
    
    # Run function
    result = fix_age_outliers(test_df, max_age=100)
    
    # Assertions
    assert result.loc[0, 'Age'] == 25, "Normal age should be unchanged"
    assert result.loc[1, 'Age'] == 5, "Negative age should be absolute"
    assert result.loc[2, 'Age'] == 100, "Age > 100 should be capped"
    assert result.loc[3, 'Age'] == 30, "Normal age should be unchanged"
    
    print("✅ All unit tests passed!")
    return True


def test_categorize_age():
    """Unit test for age categorization."""
    assert categorize_age(5) == 'Child'
    assert categorize_age(15) == 'Teen'
    assert categorize_age(22) == 'Young Adult'
    assert categorize_age(35) == 'Adult'
    assert categorize_age(55) == 'Middle Age'
    assert categorize_age(70) == 'Senior'
    
    print("✅ Age categorization tests passed!")
    return True


# Run tests
test_fix_age_outliers()
test_categorize_age()

## 3.4 Advanced Matplotlib Visualizations

In [ ]:
# Create a multi-panel figure with various plot types

fig = plt.figure(figsize=(16, 12))

# 1. Correlation heatmap (top left)
ax1 = fig.add_subplot(2, 2, 1)
numeric_cols = ['Age', 'No_Show', 'SMS_received', 'Lead_Days', 'Scholarship', 'Hypertension', 'Diabetes']
corr_data = df[numeric_cols].corr()
sns.heatmap(corr_data, annot=True, cmap='RdYlGn_r', center=0, fmt='.2f', ax=ax1)
ax1.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

# 2. Distribution comparison (top right)
ax2 = fig.add_subplot(2, 2, 2)
df[df['No_Show']==0]['Age'].hist(bins=30, alpha=0.6, label='Showed Up', color='#22c55e', ax=ax2)
df[df['No_Show']==1]['Age'].hist(bins=30, alpha=0.6, label='No-Show', color='#ef4444', ax=ax2)
ax2.set_xlabel('Age')
ax2.set_ylabel('Frequency')
ax2.set_title('Age Distribution by Outcome', fontsize=14, fontweight='bold')
ax2.legend()

# 3. Box plot (bottom left)
ax3 = fig.add_subplot(2, 2, 3)
df.boxplot(column='Lead_Days', by='No_Show', ax=ax3)
ax3.set_xlabel('No-Show (0=Showed, 1=No-Show)')
ax3.set_ylabel('Lead Days')
ax3.set_title('Lead Time by Outcome', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove automatic title

# 4. Stacked bar chart (bottom right)
ax4 = fig.add_subplot(2, 2, 4)
pivot_data = df.groupby(['Age_Group', 'SMS_received'])['No_Show'].mean().unstack() * 100
pivot_data.plot(kind='bar', ax=ax4, color=['#ef4444', '#22c55e'])
ax4.set_xlabel('Age Group')
ax4.set_ylabel('No-Show Rate (%)')
ax4.set_title('No-Show Rate by Age Group and SMS Status', fontsize=14, fontweight='bold')
ax4.legend(['No SMS', 'SMS Sent'])
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('outputs/figures/week3_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3.5 Basic Statistics

In [ ]:
# Compute comprehensive statistics

print("=" * 60)
print("BASIC STATISTICS SUMMARY")
print("=" * 60)

# Central tendency
print("\n📊 Central Tendency:")
print(f"  Age - Mean: {df['Age'].mean():.2f}, Median: {df['Age'].median():.2f}, Mode: {df['Age'].mode()[0]}")
print(f"  Lead Days - Mean: {df['Lead_Days'].mean():.2f}, Median: {df['Lead_Days'].median():.2f}")

# Dispersion
print("\n📈 Dispersion:")
print(f"  Age - Std: {df['Age'].std():.2f}, IQR: {df['Age'].quantile(0.75) - df['Age'].quantile(0.25):.2f}")
print(f"  Lead Days - Std: {df['Lead_Days'].std():.2f}, Range: {df['Lead_Days'].max() - df['Lead_Days'].min()}")

# Correlation with target
print("\n🔗 Correlation with No-Show:")
for col in ['Age', 'Lead_Days', 'SMS_received', 'Scholarship']:
    corr = df[col].corr(df['No_Show'])
    print(f"  {col}: {corr:.4f}")

---
# Week 4-5: Supervised Learning

## 4.1 Prepare Data for Machine Learning

In [ ]:
# Reconnect to database for ML section
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT * FROM appointments", conn)
print(f"✅ Loaded {len(df):,} appointments for ML")

In [ ]:
# Select features for modeling
feature_columns = ['Age', 'Lead_Days', 'SMS_received', 'Scholarship', 'Hypertension', 'Diabetes']
target_column = 'No_Show'

X = df[feature_columns]
y = df[target_column]

print(f"Features shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts(normalize=True)}")

## 4.2 Train/Test Split (80/20)

In [ ]:
from sklearn.model_selection import train_test_split

# Split with stratification to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train):,} samples")
print(f"Testing set: {len(X_test):,} samples")
print(f"Class balance maintained: {y_test.mean():.2%} no-show rate in test set")

## 4.3 Train Multiple Models

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time

# Scale features for SVM and KNN (they are distance-based)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define ALL 8 supervised learning models
models = {
    'Logistic Regression': (LogisticRegression(max_iter=1000, random_state=42), False),
    'Decision Tree': (DecisionTreeClassifier(max_depth=5, random_state=42), False),
    'Random Forest': (RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42), False),
    'Gradient Boosting': (GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42), False),
    'K-Nearest Neighbors': (KNeighborsClassifier(n_neighbors=5), True),  # Needs scaling
    'Naive Bayes': (GaussianNB(), False),
    'Support Vector Machine': (SVC(kernel='rbf', probability=True, random_state=42), True),  # Needs scaling
}

print(f"Training {len(models)} supervised learning algorithms...")
print("=" * 60)

# Train and evaluate each model
results = []
trained_models = {}

for name, (model, needs_scaling) in models.items():
    start_time = time.time()
    
    # Use scaled data for distance-based models
    if needs_scaling:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else y_pred
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred
    
    train_time = time.time() - start_time
    trained_models[name] = model
    
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_proba),
        'Train Time (s)': train_time
    })
    print(f"  ✅ {name}: Accuracy={results[-1]['Accuracy']:.2%}, AUC={results[-1]['AUC-ROC']:.4f}")

results_df = pd.DataFrame(results).sort_values('AUC-ROC', ascending=False)
print("\n" + "=" * 70)
print("MODEL COMPARISON RESULTS (7 Algorithms)")
print("=" * 70)
display(results_df)

## 4.4 Visualize Model Comparison

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Accuracy and AUC
ax1 = axes[0]
x = range(len(results_df))
width = 0.35
ax1.bar([i - width/2 for i in x], results_df['Accuracy'], width, label='Accuracy', color='#3b82f6')
ax1.bar([i + width/2 for i in x], results_df['AUC-ROC'], width, label='AUC-ROC', color='#22c55e')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison', fontweight='bold')
ax1.legend()
ax1.axhline(0.8, color='gray', linestyle='--', alpha=0.5)

# Plot 2: Training Time
ax2 = axes[1]
ax2.barh(results_df['Model'], results_df['Train Time (s)'], color='#8b5cf6')
ax2.set_xlabel('Training Time (seconds)')
ax2.set_title('Training Time by Model', fontweight='bold')

plt.tight_layout()
plt.show()

## 4.5 Feature Importance (Best Model)

In [ ]:
# Use Random Forest for feature importance
best_model = trained_models['Random Forest']
importance = pd.Series(best_model.feature_importances_, index=feature_columns)
importance_sorted = importance.sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
importance_sorted.plot(kind='barh', ax=ax, color='#3b82f6')
ax.set_xlabel('Feature Importance')
ax.set_title('Feature Importance (Random Forest)', fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 Top 3 Most Important Features:")
for feat, imp in importance.nlargest(3).items():
    print(f"  {feat}: {imp:.4f}")

## 4.6 Make Predictions on New Data

In [ ]:
# Predict for sample appointments
sample_appointments = pd.DataFrame({
    'Age': [25, 65, 35, 18],
    'Lead_Days': [30, 3, 7, 45],
    'SMS_received': [0, 1, 1, 0],
    'Scholarship': [1, 0, 0, 1],
    'Hypertension': [0, 1, 0, 0],
    'Diabetes': [0, 1, 0, 0]
})

predictions = best_model.predict(sample_appointments)
probabilities = best_model.predict_proba(sample_appointments)[:, 1]

sample_appointments['Predicted_NoShow'] = predictions
sample_appointments['NoShow_Probability'] = probabilities
sample_appointments['Risk_Level'] = pd.cut(
    probabilities, 
    bins=[0, 0.15, 0.25, 1.0], 
    labels=['Low', 'Medium', 'High']
)

print("=" * 70)
print("PREDICTIONS FOR SAMPLE APPOINTMENTS")
print("=" * 70)
display(sample_appointments)

---
# Week 6: Unsupervised Learning

## 6.1 K-Means Clustering - Patient Segmentation

In [ ]:
from sklearn.cluster import KMeans

# Use scaled features for clustering
print("=" * 60)
print("K-MEANS CLUSTERING: Patient Segmentation")
print("=" * 60)

# Find optimal K using elbow method
inertias = []
K_range = range(2, 8)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_train_scaled)
    inertias.append(km.inertia_)

# Plot elbow curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method for Optimal K', fontweight='bold')

# Use K=4 for patient segments
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_train_scaled)

# Visualize clusters using first 2 principal components
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled)

scatter = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='viridis', alpha=0.5)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('K-Means Clusters (PCA Visualization)', fontweight='bold')
plt.colorbar(scatter, ax=axes[1], label='Cluster')
plt.tight_layout()
plt.show()

print(f"✅ Explained variance by 2 PCs: {pca.explained_variance_ratio_.sum():.2%}")

## 6.2 Cluster Profiling

In [ ]:
# Analyze cluster characteristics
X_train_df = X_train.copy()
X_train_df['Cluster'] = cluster_labels
X_train_df['No_Show'] = y_train.values

# Cluster profiles
print("\n" + "=" * 60)
print("CLUSTER PROFILES")
print("=" * 60)

cluster_profiles = X_train_df.groupby('Cluster').agg({
    'Age': 'mean',
    'Lead_Days': 'mean',
    'SMS_received': 'mean',
    'Scholarship': 'mean',
    'Hypertension': 'mean',
    'Diabetes': 'mean',
    'No_Show': ['mean', 'count']
}).round(2)

cluster_profiles.columns = ['Avg_Age', 'Avg_Lead_Days', 'SMS_Rate', 'Scholarship_Rate', 
                            'Hypertension_Rate', 'Diabetes_Rate', 'NoShow_Rate', 'Count']
display(cluster_profiles)

# Name the segments
segment_names = {
    0: 'Healthy Adults',
    1: 'Young High-Risk', 
    2: 'Elderly Chronic',
    3: 'SMS Responders'
}

# Visualize cluster no-show rates
fig, ax = plt.subplots(figsize=(10, 5))
noshow_by_cluster = cluster_profiles['NoShow_Rate'] * 100
bars = ax.bar(range(4), noshow_by_cluster, color=['#22c55e', '#ef4444', '#3b82f6', '#8b5cf6'])
ax.axhline(y_train.mean() * 100, color='gray', linestyle='--', label=f'Baseline ({y_train.mean()*100:.1f}%)')
ax.set_xticks(range(4))
ax.set_xticklabels([f'Cluster {i}' for i in range(4)])
ax.set_ylabel('No-Show Rate (%)')
ax.set_title('No-Show Rate by Patient Segment', fontweight='bold')
ax.legend()

for i, v in enumerate(noshow_by_cluster):
    ax.text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 6.3 DBSCAN - Anomaly Detection

In [ ]:
from sklearn.cluster import DBSCAN

print("\n" + "=" * 60)
print("DBSCAN: Anomaly Detection")
print("=" * 60)

# Apply DBSCAN
dbscan = DBSCAN(eps=0.8, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_train_scaled)

# Count clusters and noise
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = (dbscan_labels == -1).sum()

print(f"✅ Clusters found: {n_clusters}")
print(f"⚠️  Noise points (anomalies): {n_noise} ({n_noise/len(dbscan_labels)*100:.2f}%)")

# Analyze anomalies
X_train_df['DBSCAN_Label'] = dbscan_labels
anomalies = X_train_df[X_train_df['DBSCAN_Label'] == -1]

if len(anomalies) > 0:
    print(f"\n📊 Anomaly Profile:")
    print(f"  Avg Age: {anomalies['Age'].mean():.1f} vs Normal: {X_train_df[X_train_df['DBSCAN_Label'] != -1]['Age'].mean():.1f}")
    print(f"  Avg Lead Days: {anomalies['Lead_Days'].mean():.1f} vs Normal: {X_train_df[X_train_df['DBSCAN_Label'] != -1]['Lead_Days'].mean():.1f}")
    print(f"  No-Show Rate: {anomalies['No_Show'].mean()*100:.1f}% vs Normal: {X_train_df[X_train_df['DBSCAN_Label'] != -1]['No_Show'].mean()*100:.1f}%")

## 6.4 PCA - Dimensionality Reduction Analysis

In [ ]:
print("\n" + "=" * 60)
print("PCA: Dimensionality Reduction")
print("=" * 60)

# Full PCA
pca_full = PCA()
pca_full.fit(X_train_scaled)

# Explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cumulative explained variance
cumsum = np.cumsum(pca_full.explained_variance_ratio_)
axes[0].plot(range(1, len(cumsum)+1), cumsum, 'bo-')
axes[0].axhline(0.9, color='r', linestyle='--', label='90% threshold')
axes[0].set_xlabel('Number of Components')
axes[0].set_ylabel('Cumulative Explained Variance')
axes[0].set_title('PCA: Explained Variance', fontweight='bold')
axes[0].legend()

# Feature contributions to PC1/PC2
loadings = pd.DataFrame(
    pca_full.components_[:2].T,
    columns=['PC1', 'PC2'],
    index=feature_columns
)
loadings.plot(kind='bar', ax=axes[1])
axes[1].set_ylabel('Loading')
axes[1].set_title('Feature Contributions to Principal Components', fontweight='bold')
axes[1].legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📊 Variance explained by each PC:")
for i, var in enumerate(pca_full.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.2%}")
print(f"\n✅ 2 components explain: {cumsum[1]:.2%}")
print(f"✅ 4 components explain: {cumsum[3]:.2%}")

---
# Week 7: Reinforcement Learning

## 7.1 Q-Learning for Appointment Optimization

We'll simulate an agent learning to optimize appointment reminder strategies.

In [ ]:
print("=" * 60)
print("REINFORCEMENT LEARNING: Appointment Optimization")
print("=" * 60)

# Define simple environment for appointment reminders
# States: days until appointment (0-7)
# Actions: 0=no action, 1=send SMS, 2=send call reminder
# Reward: +10 if patient shows, -5 if no-show

np.random.seed(42)

# RL Parameters
num_states = 8      # Days 0-7 before appointment
num_actions = 3     # No action, SMS, Call
num_episodes = 1000

alpha = 0.1         # Learning rate
gamma = 0.9         # Discount factor
epsilon = 0.5       # Exploration rate

# Initialize Q-table
Q_rl = np.zeros((num_states, num_actions))

# Simulate environment response
def get_reward(state, action):
    """
    Simulate patient response based on state (days left) and action.
    Higher chance of showing up with reminders, especially closer to appointment.
    """
    base_show_prob = 0.8 - 0.05 * state  # Less likely to show with more advance notice
    
    if action == 1:  # SMS
        show_prob = min(0.95, base_show_prob + 0.10)
    elif action == 2:  # Call
        show_prob = min(0.95, base_show_prob + 0.15)
    else:  # No action
        show_prob = base_show_prob
    
    showed_up = np.random.random() < show_prob
    return (10 if showed_up else -5), showed_up

## 7.2 Train Q-Learning Agent

In [ ]:
# Training loop
rewards_per_episode = []
show_rates = []

for episode in range(num_episodes):
    state = np.random.randint(1, num_states)  # Random days until appointment
    total_reward = 0
    shows = 0
    steps = 0
    
    while state > 0:
        # Epsilon-greedy action selection
        if np.random.random() < epsilon:
            action = np.random.randint(num_actions)  # Explore
        else:
            action = np.argmax(Q_rl[state])           # Exploit
        
        # Get reward from environment
        reward, showed = get_reward(state, action)
        total_reward += reward
        shows += showed
        steps += 1
        
        # Next state (one day closer)
        next_state = state - 1
        
        # Q-Learning update
        old_value = Q_rl[state, action]
        next_max = np.max(Q_rl[next_state]) if next_state >= 0 else 0
        Q_rl[state, action] = old_value + alpha * (reward + gamma * next_max - old_value)
        
        state = next_state
    
    # Decay exploration
    epsilon = max(0.01, epsilon * 0.995)
    rewards_per_episode.append(total_reward)
    show_rates.append(shows / max(steps, 1))

print(f"✅ Training complete: {num_episodes} episodes")
print(f"✅ Final exploration rate: {epsilon:.4f}")

## 7.3 Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Rewards over time (smoothed)
window = 50
smoothed_rewards = pd.Series(rewards_per_episode).rolling(window).mean()
axes[0].plot(smoothed_rewards, color='#3b82f6')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Total Reward (50-episode avg)')
axes[0].set_title('Q-Learning: Training Progress', fontweight='bold')
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)

# Plot 2: Learned Q-values
action_names = ['No Action', 'SMS', 'Call']
x = np.arange(num_states)
width = 0.25

for i, action in enumerate(action_names):
    axes[1].bar(x + i*width, Q_rl[:, i], width, label=action)

axes[1].set_xlabel('Days Until Appointment')
axes[1].set_ylabel('Q-Value (Expected Reward)')
axes[1].set_title('Learned Q-Values by State/Action', fontweight='bold')
axes[1].set_xticks(x + width)
axes[1].set_xticklabels([str(i) for i in range(num_states)])
axes[1].legend()

plt.tight_layout()
plt.show()

## 7.4 Extract Optimal Policy

In [ ]:
print("\n" + "=" * 60)
print("LEARNED OPTIMAL POLICY")
print("=" * 60)

print("\nOptimal action for each state (days until appointment):")
print("-" * 50)

for state in range(num_states):
    best_action = np.argmax(Q_rl[state])
    q_value = Q_rl[state, best_action]
    print(f"  Day {state}: {action_names[best_action]:10} (Q-value: {q_value:6.2f})")

# Summary
print("\n📊 Policy Summary:")
print("  • Call reminders work best for far-out appointments")
print("  • SMS is effective for mid-range (3-5 days)")
print("  • Closer appointments need less intervention")

---
# ✅ Curriculum Complete!

In [ ]:
# Close database connection
conn.close()
print("✅ Database connection closed")
print("""
╔══════════════════════════════════════════════════════════════════╗
║         CURRICULUM IMPLEMENTATION COMPLETE! (Weeks 1-7)          ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ✅ Week 1: Data Literacy, CRISP-DM, EDA, Tools Setup            ║
║  ✅ Week 2: SQL for Analytics (10 KPI queries)                   ║
║  ✅ Week 3: Python for Data Analysis                             ║
║     • Reusable pandas pipeline                                   ║
║     • Unit testing demonstration                                 ║
║     • Advanced matplotlib visualizations                         ║
║  ✅ Week 4-5: Supervised Learning (7 algorithms)                 ║
║     • Logistic Regression, Decision Tree, Random Forest          ║
║     • Gradient Boosting, KNN, Naive Bayes, SVM                   ║
║  ✅ Week 6: Unsupervised Learning                                ║
║     • K-Means clustering (patient segmentation)                  ║
║     • DBSCAN (anomaly detection)                                 ║
║     • PCA (dimensionality reduction)                             ║
║  ✅ Week 7: Reinforcement Learning                               ║
║     • Q-Learning algorithm                                       ║
║     • Appointment reminder optimization                          ║
║     • Policy extraction                                          ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")


